# JIT(Just In Time) 컴파일

본 글에서는 Jax가 어떻게 동작하고, 어떻게 기존 python보다 높은 성능을 이끌어 낼 수 있는지를 삺펴보겠습니다. 특히 XLA를 사용하여 파이썬 함수를 JIT 컴파일 시켜주는, `jax.jit()` 함수에 대해서 알아보겠습니다.

## Jax 변환은 어떻게 작동하는가

이전 글에서 우리는 파이썬 함수를 `jaxpr` 형식으로 변환시켜 보았습니다. Jax는 함수의 각 연산들을 primitive 연산으로 변환시키게 됩니다. primitive 연산은 각각 매우 근본적인 컴퓨터 연산자와 대응됩니다.

`jax.make_jaxpr()` 를 통해 함수의 primitive 연산을 확인할 수 있습니다:

In [2]:
import jax
import jax.numpy as jnp

global_list = []

def log2(x):
    global_list.append(x)
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2

print(jax.make_jaxpr(log2)(3.0))

{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


위 예제에서 알 수 있는 매우 중요한 사실은 `Jaxpr`는 **부수효과**를 무시한다는 것입니다.
**부수효과**란 함수형 프로그래밍에서 사용되는 단어로, 함수의 아웃풋에 영향을 끼치지 않는 코드 실행을 뜻합니다(영어로는 side-effect라고 합니다). 예를 들어 위 코드에서 `global_list.append(x)`는 실제로는 중요한 역할을 하고 있을 수도 있으나, 함수의 리턴값에 영향을 미치지 않기 때문에 `Jaxpr`에는 무시되었습니다. 이 코드는 부수효과를 일으키는 코드라 할 수 있습니다.

부수효과를 무시하는 것은 버그가 아닌 Jax의 기능입니다.부수효과가 없는 코드를 작성하는 것은 결과적으로 보이지 않는 오류의 발생을 줄이는 길이며, 이는 Jax 라이브러리의 철학입니다.

부수효과가 있는 함수는 Jax 변환이 일어날 때 정상적인 작동이 되지 않을 수도 있으므로 위험한 존재입니다. 이는 쥐도새도 모르게 오류를 일으킬 수도 있으며, 혹은 Tracer 누수와 같은 문제를 일으킬 수도 있습니다. 더욱이 Jax는 이러한 부수효과가 존재하는지 조차 인지할 수 없습니다(jax 상에서 디버깅 출력을 원한다면 [`jax.debug.print()`]('https://jax.readthedocs.io/en/latest/_autosummary/jax.debug.print.html#jax.debug.print)를, 누수 Tracer를 확인하고 싶다면 [`jax.check_tracer_leaks()`](https://jax.readthedocs.io/en/latest/_autosummary/jax.check_tracer_leaks.html#jax.check_tracer_leaks)를 사용할 수 있습니다).

Tracing을 할 떄, Jax는 `jax.Array`들을 tracer로 대체합니다(정확히는 tracer로 wrapping합니다). 이 tracer들은 자신에게 일어나는 Jax 연산을 모두 기록하고, 이 기록을 통해 함수를 재구현하게 됩니다. 재구현의 결과가 바로 `jaxpr`입니다. tracer는 부수효과를 기록하지 않기 때문에 `jaxpr`에는 부수효과가 나타나지 않습닌다. 하지만 tracing을 할 때에는 부수효과가 일어납니다.

주의: 파이썬의 `print()`는 부수효과를 발생시키는 함수입니다. 힘수의 아웃풋에 영향을 미치지 않기 때문입니다. 고로 `print()` 함수는 tracing 과정에서는 실행되지만, jaxpr 때에는 나타나지 않습니다:

In [2]:
def log2_with_print(x):
    print(f"printed x: {x}")
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2

print(jax.make_jaxpr(log2_with_print)(3.))

printed x: Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>
{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


Tracing 과정에서 `print()` 함수를 통해 출력된 변수 x 는 3. 이 아닌 traced 타입을 가지고 있는 것을 알 수 있습니다.

여기서 알 수 있는 중요한 사실은: jaxpr은 실제로 실행이 일어난 연산에 대해서만 기록된다는 것입니다. 예를 들어, 아래와 같은 함수를 jaxpr로 변환할 시에는 실제로 일어난 조건에 대해서만 연산이 기록되게 됩니다:

In [3]:
def log2_if_rank_2(x):
  if x.ndim == 2:
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2
  else:
    return x

print(jax.make_jaxpr(log2_if_rank_2)(jax.numpy.array([1, 2, 3])))

{ lambda ; a:i32[3]. let  in (a,) }


## 함수를 JIT 컴파일 하기

JAX는 코드 하나 바꾸지 않고 CPU/GPU/TPU 에서 모두 사용할 수 있습니다. 딥러닝에서 자주 사용되는 함수인 SELU 함수를 예로 보겠습니다:

In [4]:
import jax
import jax.numpy as jnp

def selu(x, alpha=1.67, lambda_=1.05):
  return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(1000000)
%timeit selu(x).block_until_ready()

1.78 ms ± 64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


위 코드는 selu 함수 내부에 일어나는 매 연산마다 연산장치(CPU/GPU/TPU)에 연산을 요청하게 됩니다. 이는 속도를 저하시키는 요인이 됩니다.

우리는 XLA 컴파일러에게 코드에 대해 최대한 많은 정보를 줌으로써 전체적인 코드를 최적화하고 실행 속도를 향상시키고 싶습니다. 이는 JIT 컴파일을 통해 이뤄질 수 있으며, `jax.jit()`함수는 그 역할을 수행합니다. 아래 예시는 JIT을 통해 selu 함수의 속도를 향상시키는 방법을 보여줍니다:

In [5]:
selu_jit = jax.jit(selu)

# 시간 측정 하기 전 먼저 컴파일 시키기 위한 단계
selu_jit(x).block_until_ready()

%timeit selu_jit(x).block_until_ready()

670 µs ± 48.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)



1. 우리는 `jax.jit()`을 통해 `selu`를 컴파일한 함수인 `selu_jit`를 만들었습니다.
2. 두 번째 코드에서 x에 대해 selu_jit을 처음 호출했습니다. 여기서 tracing이 일어납니다. Tracing을 하기 위해서는 tracer가 필요하고, tracer는 결국 배열을 변환(wrapping)하면서 얻어지는 것이므로, 결국 어떠한 배열이 들어와야 tracing이 일어날 수 있습니다. Tracing을 통해, GPU나 TPU에 최적화된 XLA로 작동되는 jaxpr이 얻어지게 됩니다. 그리고 이후 함수를 호출하면 jaxpr 형태의 JIT 컴파일된 함수가 호출되게 됩니다. 마지막 줄에서 호출되는 `selu_jit` 은 파이썬 구현체를 완전히 무시한 체 컴파일된 함수를 그대로 사용하게 됩니다. (첫 실행 때의 시간을 측정하게 되면 컴파일된 함수의 속도가 아닌, 파이썬 함수를 실행사는 속도 + 트레이싱이 진행되는 속도가 나오게 될 것입니다.)
3. 최종적으로 컴파일된 `selu_jit` 함수를 실행하여 시간을 측정합니다(JAX 연산은 파이썬과 비동기적으로 일어나므로 `block_until_ready()`함수를 통해 시간을 측정해줘야 합니다).